In [1]:
# Тюнинг
import optuna as opt
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.metrics import f1_score

# Данные
import os
import pandas as pd

In [2]:
# Пути
ROOT = os.getcwd()
TRAIN_DATASET = os.path.join(ROOT, '../data/train_AIC.csv')
TEST_DATASET = os.path.join(ROOT, '../data/test_AIC.csv')

# Загрузка
train_df = pd.read_csv(TRAIN_DATASET)
test_df = pd.read_csv(TEST_DATASET)

X, y = train_df.iloc[:, :-1], train_df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [7]:
# Функция оптимизации
def objective(trial: opt.Trial):
    # Параметры
    learning_rate = trial.suggest_float('learning_rate', 0.1, 0.7, log=True)
    n_estimators = trial.suggest_int('n_estimators', 100, 1000, 50)
    max_depth = trial.suggest_int('max_depth', 4, 12)
    max_bin = trial.suggest_int('max_bin', 32, 128)
    l2_leaf_reg = trial.suggest_float('l2_reg', 0.1, 1, log=True)

    # Модель
    model = CatBoostClassifier(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        l2_leaf_reg=l2_leaf_reg
        )

    cv_score = cross_val_score(model, X_train, y_train, cv=StratifiedKFold(), scoring='f1_macro', n_jobs=-1)
    accuracy = cv_score.mean()
    return accuracy

In [13]:
study = opt.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2023-08-06 14:28:49,649] A new study created in memory with name: no-name-1b05bebf-aad2-4bad-ae1f-053b7e917848
[I 2023-08-06 14:29:27,957] Trial 0 finished with value: 0.8009251192755394 and parameters: {'learning_rate': 0.49446996907119856, 'n_estimators': 150, 'max_depth': 6, 'max_bin': 111, 'l2_reg': 0.968319904071682}. Best is trial 0 with value: 0.8009251192755394.
[W 2023-08-06 14:34:03,452] Trial 1 failed with parameters: {'learning_rate': 0.1753768383441368, 'n_estimators': 500, 'max_depth': 10, 'max_bin': 117, 'l2_reg': 0.13482034440577745} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "o:\Programing\predict-supply-disruption\env\Lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\Dead4an\AppData\Local\Temp\ipykernel_7600\2425225681.py", line 18, in objective
    cv_score = cross_val_score(model, X_train, y_train, cv=Stratif

KeyboardInterrupt: 